In [1]:
import tensorflow as tf
import numpy as np
import os
import re
import io


In [46]:
import pickle
data_dir = "/home/maikfangogoair/tmp/save/"
pkl_file = "spect_v1.pkl"
with open (data_dir + pkl_file, 'rb') as fp:
    all_data = pickle.load(fp)

In [7]:
tf.reset_default_graph();
input=tf.placeholder(tf.float32, (None, 99, 161), 'input')
def spectrogram_ConvNet(input):
    """
    """
    dropout_prob = tf.placeholder(tf.float32, name='dropout_prob')
    
    
    reshape_input = tf.reshape(input, [-1, 99, 161, 1], 'reshape_input')
    batch_input = tf.layers.batch_normalization(reshape_input)
    
    conv_1 = tf.layers.conv2d(batch_input, 64, (5,5), (1,1), 'same', name='conv_1')
    relu_1 = tf.nn.relu(conv_1,'relu_1')
    dropout_1 = tf.nn.dropout(relu_1, dropout_prob, name = 'dropout_1')
    
    max_pool = tf.nn.max_pool(dropout_1, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')

    conv_2 = tf.layers.conv2d(max_pool, 64, (3,3), (1,1), 'same')
    relu_2 = tf.nn.relu(conv_2,'relu_2')
    dropout_2 = tf.nn.dropout(relu_2, dropout_prob, name = 'dropout_2')
    
    size=int(dropout_2.get_shape()[1]) * int(dropout_2.get_shape()[2]) * int(dropout_2.get_shape()[3])
    flatten = tf.reshape(dropout_2, (-1, size), 'flatten')

    logits = tf.layers.dense(flatten, 12,name = 'logits')
    
    return logits,dropout_prob

In [56]:
tf.reset_default_graph();
input=tf.placeholder(tf.float32, (None, 1, 161, 101))
              
def VGG11(input, final_class, verbose):
    steps = [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']
    dropout_prob = tf.placeholder(tf.float32, name='dropout_prob')
    features = tf.transpose(input,[0, 2, 3, 1])
    if verbose: print(features)
    for i in range(len(steps)):
        step = steps[i]
        with tf.variable_scope("Layer"+str(i+1), reuse=False):
            if step == 'M':
                features = tf.layers.max_pooling2d(features , (2,2) , (2,2) ,"same")
                if verbose: print(features)
            else:
                features = tf.layers.conv2d(features, step, (3,3), (1,1), 'same', name='conv_1')
                if verbose: print(features)
                features = tf.layers.batch_normalization(features)
                if verbose: print(features)
                features = tf.nn.relu(features)
                if verbose: print(features)
    features = tf.layers.average_pooling2d(features , (1,1), (1,1), 'same')
    if verbose: print(features)
    features = tf.layers.flatten(features)
    if verbose: print(features)
    features = tf.layers.dense(features, 512)
    if verbose: print(features)
    logits = tf.layers.dense(features, final_class )
    if verbose: print(features)
    
    return logits,dropout_prob

In [57]:
model_name = "VGG11"
model_cfg = {"spectrogram_ConvNet": spectrogram_ConvNet, "VGG11": VGG11}
model = model_cfg[model_name]
final_class = 30
logits,dropout_prob = model(input, final_class, False)

In [60]:
#training steps
ground_truth_input = tf.placeholder(tf.int64, [None], name='groundtruth_input')
learning_rate_input = tf.placeholder(tf.float32, [], name='learning_rate_input')
cross_entropy_mean = tf.losses.sparse_softmax_cross_entropy(ground_truth_input, logits)
#train_step = tf.train.GradientDescentOptimizer(learning_rate_input).minimize(cross_entropy_mean)
train_step = tf.train.AdamOptimizer(learning_rate_input).minimize(cross_entropy_mean)

predicted_indices = tf.argmax(logits, 1)
correct_prediction = tf.equal(predicted_indices, ground_truth_input)
confusion_matrix = tf.confusion_matrix(ground_truth_input, predicted_indices, num_classes=12)
evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
import random
random.seed(1222)
size=len([x for x in all_data if x["group"] == "training" ])
saver = tf.train.Saver(tf.global_variables())
init = tf.global_variables_initializer()

batch_size = 100
learning_rate = 1e-3
learning_decay = 0.9
learning_decay_period = 1000
eval_every_steps = 1000
training_steps = 20000
print_every = 10
eval_size = 100
validation_size = len([x["label_idx"] for x in all_data if x["group"] == "validation"])
print("validation set count:" + str(validation_size))

with tf.Session() as sess:
    sess.run(init)
    #saver.restore(sess, data_dir + 'spectrogram_ConvNet.ckpt-700')
    for i in range(1, training_steps+1):
        start_pos = (i-1) * batch_size % size
        end_pos = (i-1) * batch_size % size + batch_size 
        train_fingerprints = np.asarray([x["feature"] for x in all_data if x["group"] == "training"][start_pos:end_pos])
        train_ground_truth = np.asarray([x["label_idx"] for x in all_data if x["group"] == "training"][start_pos:end_pos]) 
        loss, _ ,accurancy,  = sess.run([cross_entropy_mean, train_step, evaluation_step], \
                feed_dict={
                    input: train_fingerprints,
                    ground_truth_input: train_ground_truth,
                    learning_rate_input: learning_rate * (learning_decay**(i // learning_decay_period)),
                    dropout_prob: 0.5
                })
        if i % print_every == 0:
            print("Step %d, loss: %f, accurancy: %f." % (i, loss, accurancy))
        if  i % eval_every_steps == 0:
            saver.save(sess, data_dir + model_name +".ckpt", global_step=i)
            rand_choice = [random.randint(0,validation_size-1) for x in range(eval_size)]
            _, accurancy = sess.run([train_step,evaluation_step], \
                feed_dict={
                    input: np.asarray([x["feature"] for x in all_data if x["group"] == "validation"])[rand_choice],
                    ground_truth_input: np.asarray([x["label_idx"] for x in all_data if x["group"] == "validation"])[rand_choice], \
                    learning_rate_input: learning_rate * (learning_decay**(i // learning_decay_period)),
                    dropout_prob: 1.0
                })
            print("Validation accurancy is %f" % accurancy)

validation set count:6798
Step 10, loss: 3.396253, accurancy: 0.050000.
Step 20, loss: 3.377050, accurancy: 0.060000.
Step 30, loss: 3.237314, accurancy: 0.050000.
Step 40, loss: 2.989235, accurancy: 0.170000.
Step 50, loss: 2.843735, accurancy: 0.160000.
Step 60, loss: 2.320439, accurancy: 0.360000.
Step 70, loss: 2.692460, accurancy: 0.300000.
Step 80, loss: 1.835875, accurancy: 0.440000.
Step 90, loss: 1.621885, accurancy: 0.520000.
Step 100, loss: 1.549716, accurancy: 0.510000.
Step 110, loss: 1.153748, accurancy: 0.660000.
Step 120, loss: 0.887473, accurancy: 0.730000.
Step 130, loss: 0.998535, accurancy: 0.680000.
Step 140, loss: 1.051070, accurancy: 0.660000.
Step 150, loss: 1.405718, accurancy: 0.640000.
Step 160, loss: 1.043543, accurancy: 0.700000.
Step 170, loss: 0.920910, accurancy: 0.710000.
Step 180, loss: 1.006861, accurancy: 0.730000.
Step 190, loss: 0.805675, accurancy: 0.800000.
Step 200, loss: 0.633963, accurancy: 0.760000.
Step 210, loss: 0.901938, accurancy: 0.7500

In [14]:
#testing
with tf.Session() as sess:
    saver = tf.train.Saver()
    saver.restore(sess, data_dir + 'spectrogram_ConvNet.ckpt-1600')
    predicted_list = sess.run(predicted_indices, \
        feed_dict={
            input: np.asarray([x["spectrogram"] for x in all_data if x["group"] == "testing"]) ,
            dropout_prob: 1.0
        })

INFO:tensorflow:Restoring parameters from /home/maikfangogoair/tmp/save/spectrogram_ConvNet.ckpt-1600


INFO:tensorflow:Restoring parameters from /home/maikfangogoair/tmp/save/spectrogram_ConvNet.ckpt-1600


In [51]:
predicted_list_part = np.hstack((predicted_list_part1,predicted_list_part2,predicted_list_part3))

predicted_list_part

correct_idx = np.asarray([x["label_idx"] for x in all_data if x["group"] == "testing"])

wrong_cnt = 0
for i in range(correct_idx.shape[0]):
    if predicted_list_part[i] != correct_idx[i]:
        wrong_cnt += 1
wrong_cnt

(2979 - 1255)/2979

test_data = [x for x in all_data if x["group"] == "testing"]
test_data[1]

wrong_data = []
word_list = ['silence','unknown', 'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
for i in range(correct_idx.shape[0]):
    if predicted_list_part[i] != correct_idx[i]:
        test_data[i]["predicted_label"] = word_list[predicted_list_part[i]]
        wrong_data.append(test_data[i])

#which pair most confusing?
confuse_map = {}
for x in wrong_data:
    key = x["label"] + " is confused by " + x["predicted_label"]
    cnt = confuse_map.get(key, 0)
    confuse_map[key] = cnt + 1 
#https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value
import operator
sorted_confuse_map = sorted(confuse_map.items(), key=operator.itemgetter(1), reverse=True)
sorted_confuse_map